# Reinforcement Learning - Frozenlake

Before diving into the topic of **Deep Reinforcement** it is important to get to know the basic concepts of Reinforcement Learning as this allows us to improve our understanding of the whole field.

To do so we take a look at one of the most known classical Reinforcement Learning algorithm called **Q-Learning** and apply this algorithm on the [FrozenLake environment](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/).

<img src="./images/frozenlake.gif" width="200px" />

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
desc=["SFFF", "FHFH", "FFFH", "HFFG"]
env = gym.make('FrozenLake-v1', desc=desc, map_name="4x4", is_slippery=False)

`is_slippery`: True/False. If True will move in intended direction with probability of 1/3 else will move in either perpendicular direction with equal probability of 1/3 in both directions.

For example, if action is left and is_slippery is True, then:
- P(move left)=1/3
- P(move up)=1/3
- P(move down)=1/3

In [ ]:
env.reset()  # Reset to initial state
for _ in range(5):
    env.render()  # Render on the screen
    action = env.action_space.sample()  # chose a random action
    env.step(action)  # Perform random action on the environment
env.close()  # dont forget to close the environment

## Seeing ASCII movement  with Python

In [ ]:
import time
from IPython.display import clear_output
for letter in ['a','b','c','d','e']:
    print(letter)
    time.sleep(1)
    clear_output(wait=True)

## ASCII Visualization of FrozenLake

In [ ]:
observation = env.reset()  # Reset to initial state
for epsiode in range(15):
    print(observation)
    env.render()  # Render on the screen
    time.sleep(0.2)
    clear_output(wait=True)
    action = env.action_space.sample()  # chose a random action
    observation, reward, done, info = env.step(action)  # Perform random action on the environment
    if done:
        env.reset()
    
env.close()  # dont forget to close the environment

## FrozenLake: Action Space

In [ ]:
env.action_space

The agent at each tile has one of 4 actions:
- 0: LEFT
- 1: DOWN
- 2: RIGHT
- 3: UP


## FrozenLake: Observation Space

The observation is a value representing the agent’s current position as `current_row * nrows + current_col` (where both the row and col start at 0). For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15. 

The number of possible observations is dependent on the size of the map. For example, the 4x4 map has 16 possible observations.  An observation is a single digit representing the current location, for a 4by4 "lake":

     0   1   2   3
     4   5   6   7
     8   9  10  11
    12  13  14  15

In [ ]:
env.observation_space

# FrozenLake: Reward

The agent is given a reward depending on whether it arrived at the "goal" tile or not:
- Reached Goal (G): +1
- Reached Hole (H): 0
- Reached Frozen (F): 0

## Manual Playing FrozenLake

In [ ]:
def asdw():
    '''
    This function gets the key press for gym action choice
    '''
    k = input()
    if k == 'a':
        action = 0
    if k == 's':
        action = 1
    if k == 'd':
        action = 2
    if k == 'w':
        action = 3
        
    return action

In [ ]:
env = gym.make('FrozenLake-v1', desc=desc, map_name="4x4", is_slippery=False)
env.reset()  # Reset to initial state
done = False
print(f"Press: a=left,w=up,s=down,d=right")
for _ in range(10):
    if not done:
        env.render()  # Render on the screen
        clear_output(wait=True)
        action = asdw()  # chose an action
        observation, reward, done, info = env.step(action)  # Perform random action on the environment
        print(f"state={observation}, reward={reward}, done={done}")
env.close()  # dont forget to close the environment
print(f"Game over")

## Q-Table : valuing each step of the agent in the environment

A "Q-Table" is essentially a mapping of all possible `state,action` pairs and the expected reward for taking an action at a particular state that we will keep updating.

$$Q(s_t,a_t)$$

For our simple discrete Frozen Lake problem, this means we have 4 actions for columns, and 16 possible states (player location on the 4 by 4 grid). So our table will look like:

<table style="width:100%">
  <tr>
      <th></th>
    <th>A0 - LEFT</th>
    <th>A1 - DOWN</th>
    <th>A2 - RIGHT</th>
    <th>A3 - UP</th>
  </tr>
  <tr>
    <td><strong>State 0</strong></td>
    <td>Q(s,a)</td>
    <td>Q(s,a)</td>
      <td>Q(s,a)</td>
      <td>Q(s,a)</td>
  </tr>
  <tr>
      <td><strong>State 1</strong></td>
    <td>Q(s,a)</td>
    <td>Q(s,a)</td>
    <td>Q(s,a)</td>
      <td>Q(s,a)</td>
  </tr>
    <tr>
      <td><strong>State ...</strong></td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
        <td>...</td>
  </tr>
    <tr>
      <td><strong>State 15</strong></td>
    <td>Q(s,a)</td>
    <td>Q(s,a)</td>
    <td>Q(s,a)</td>
        <td>Q(s,a)</td>
  </tr>
</table>

In [ ]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [ ]:
# let's initialize our Q-table of values with no prior knowledge about the game:
q_table = np.zeros([state_size, action_size])
q_table

## Learning the Q-Table

The Q-Learning update functions will require hyperparameters. we'll define them here. Often the best place to choose a good starting value is reading publications or through experimentation. Unfortunately, its very difficult to give general advice, as most environments are radically different to each other, and often hyperparameter tuning is required.

Q-Learning Equation Parameters: [Wikipedia](https://en.wikipedia.org/wiki/Q-learning)

In [ ]:
EPOCHS=20000       # number of epochs/episodes to train for
ALPHA = 0.8        # the learning rate
GAMMA = 0.95       # the discount rate
MAX_EPISODES = 100 

## $\epsilon$-Learning : exploring vs exploitation

We need to tune how quickly we "explore" the parameter space vs how quickly we "exploit" what we have learned to get to the goal.
- A random number of times ($0.0 \le \epsilon \le 1.0$), we want to randomly choose a move.
- The other times with probability $(1 - \epsilon)$, we choose the best action for a given state (row)

Tuning $\epsilon$ over time is a hyperparameter that we need to decide on. Reduce too fast, agent won't have enough time to learn. Reduce too slow, you're wasting time picking random actions. Key here is that these value help balance exploration (random choice) versus explotation (always picking what works for that Q(s,a). It's a tough balance!

In [ ]:
# Exploration vs. Exploitation parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001             # Exponential decay rate for exploration prob

## Q-Table Update (aka Learning)

Now it is time to dive into the training / Q-Table update methodology.
First we will define some functions needed for training phase

- `epsilon_greedy_action_selection`: Is used to implement the epsilon greedy action selection routine.
- `compute_next_q_value`: Computes the next Q-Values according to the formula from the lecture
- `reduce_epsilon`: Reduces the  𝜖  used for the epsilon greedy algorithm

### Selecting an Action with $\epsilon$-greedy Strategy

If we simply always select the `argmax()` qtable value during training, we'll most likely get stuck in an exploitation loop, so we'll use a random value to randomly select an action from time to time, helping the model explore , rather than exploit.

In [ ]:
def epsilon_greedy_action_selection(epsilon, q_table, discrete_state):
    '''
    Returns an action for the agent. Note how it uses a random number to decide on
    exploration versus explotation trade-off.
    '''
    random_number = np.random.random()
    
    # EXPLOITATION, USE BEST Q(s,a) Value
    if random_number > epsilon:
        # Action row for a particular state
        state_row = q_table[discrete_state,:]
        # Index of highest action for state
        # Recall action is mapped to index (e.g. 0=LEFT, 1=DOWN, etc..)
        action = np.argmax(state_row)
    
    # EXPLORATION, USE A RANDOM ACTION
    else:
        # Return a random 0,1,2,3 action
        action = env.action_space.sample()
        
    return action

### Computing Q-Value

Here we have our main Q-Learning update equation, note how it takes in the old q-value, the next optimal q value, along with our current reward, and then updates the next q value accordingly.

In [ ]:
def compute_next_q_value(old_q_value, reward, next_optimal_q_value):
    return old_q_value +  ALPHA * (reward + GAMMA * next_optimal_q_value - old_q_value)

### Reducing Epsilon

As training continues, we need to balance explotation versus exploration, we want ot make sure our agent doesn't get trapped in a cycle going from an F square to another F Square back and forth. We also don't want our agent permanently choosing random values. We'll use the function below to try to balance this.

In [ ]:
def reduce_epsilon(epsilon,epoch):
    return min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*epoch)

### Let's TRAIN!

In [ ]:
q_table = np.zeros([state_size, action_size])
total_reward = 0
epsilon = 1

In [ ]:
# Keep track of the rewards to see progress later
rewards = []
for episode in range(EPOCHS):
    # Reset the environment
    state = env.reset()
    done = False
    total_rewards = 0
    
    while not done:
        action = epsilon_greedy_action_selection(epsilon,q_table, state)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Look up current/old qtable value Q(s_t,a_t)
        old_q_value =  q_table[state,action]  

        # Get the next optimal Q-Value
        next_optimal_q_value = np.max(q_table[new_state, :])  

        # Compute next q value
        next_q = compute_next_q_value(old_q_value, reward, next_optimal_q_value)   

        # Update Q Table
        q_table[state,action] = next_q
        
        total_rewards = total_rewards + reward
        
        # Our new state is state
        state = new_state
        
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = reduce_epsilon(epsilon,episode) 
    rewards.append(total_rewards)

env.close()

In [ ]:
plt.plot(range(EPOCHS),np.cumsum(rewards))

In [ ]:
q_table

## Playing with Learned Q-Table

In [ ]:
state = env.reset()
done = False
for _ in range(10):    
    if not done:
        env.render()
        action = np.argmax(q_table[state])  # and chose action from the Q-Table
        state, reward, done, info = env.step(action) # Finally perform the action

        time.sleep(1)
        clear_output(wait=True)

env.close()
print(f"Game over")